In [36]:
import numpy as np

# Import data

X_train = np.array(list(np.loadtxt("train3.csv", delimiter=","))).astype("float32")
X_test = np.array(list(np.loadtxt("eval3.csv", delimiter=","))).astype("float32")
y_train = np.array(list(np.loadtxt("train_label3.csv", delimiter=","))).astype("float32")
y_test = np.array(list(np.loadtxt("eval_label3.csv", delimiter=","))).astype("float32")

print(np.shape(X_train),np.shape(X_test),np.shape(y_train),np.shape(y_test))

(7200, 1728) (1800, 1728) (180, 40) (180, 10)


In [37]:
y_train=np.reshape(y_train,(-1,1))
y_test=np.reshape(y_test,(-1,1))

In [38]:
# Scale data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
'''
scaler.fit(y_train)
y_train = scaler.transform(y_train)
y_test = scaler.transform(y_test)
# Build X and y
X_train = data_train[:, 1:]
y_train = data_train[:, 0]
X_test = data_test[:, 1:]
y_test = data_test[:, 0]'''

'\nscaler.fit(y_train)\ny_train = scaler.transform(y_train)\ny_test = scaler.transform(y_test)\n# Build X and y\nX_train = data_train[:, 1:]\ny_train = data_train[:, 0]\nX_test = data_test[:, 1:]\ny_test = data_test[:, 0]'

In [39]:
# Import TensorFlow
import tensorflow as tf

# Model architecture parameters
n_input = 1728
n_neurons_1 = 2048
n_neurons_2 = 1024
n_neurons_3 = 512
n_neurons_4 = 256
n_target = 1

# Placeholder
X = tf.placeholder(dtype=tf.float32, shape=[None, n_input])
Y = tf.placeholder(dtype=tf.float32, shape=[None,1])

In [40]:
# Initializers
sigma = 1
weight_initializer = tf.variance_scaling_initializer(mode="fan_avg", distribution="uniform", scale=sigma)
bias_initializer = tf.zeros_initializer()

In [41]:
# Layer 1: Variables for hidden weights and biases
W_hidden_1 = tf.Variable(weight_initializer([n_input, n_neurons_1]))
bias_hidden_1 = tf.Variable(bias_initializer([n_neurons_1]))
# Layer 2: Variables for hidden weights and biases
W_hidden_2 = tf.Variable(weight_initializer([n_neurons_1, n_neurons_2]))
bias_hidden_2 = tf.Variable(bias_initializer([n_neurons_2]))
# Layer 3: Variables for hidden weights and biases
W_hidden_3 = tf.Variable(weight_initializer([n_neurons_2, n_neurons_3]))
bias_hidden_3 = tf.Variable(bias_initializer([n_neurons_3]))
# Layer 4: Variables for hidden weights and biases
W_hidden_4 = tf.Variable(weight_initializer([n_neurons_3, n_neurons_4]))
bias_hidden_4 = tf.Variable(bias_initializer([n_neurons_4]))

# Output layer: Variables for output weights and biases
W_out = tf.Variable(weight_initializer([n_neurons_4, n_target]))  #notice the change
bias_out = tf.Variable(bias_initializer([n_target]))

In [42]:
# Hidden layer
hidden_1 = tf.nn.leaky_relu(tf.add(tf.matmul(X, W_hidden_1), bias_hidden_1))
hidden_2 = tf.nn.leaky_relu(tf.add(tf.matmul(hidden_1, W_hidden_2), bias_hidden_2))
hidden_3 = tf.nn.leaky_relu(tf.add(tf.matmul(hidden_2, W_hidden_3), bias_hidden_3))
hidden_4 = tf.nn.leaky_relu(tf.add(tf.matmul(hidden_3, W_hidden_4), bias_hidden_4))

# Output layer (must be transposed)
out = tf.transpose(tf.add(tf.matmul(hidden_4, W_out), bias_out)) #notice the change

In [46]:
# Cost function
mse = tf.losses.log_loss(np.abs(Y)+0.001, np.abs(out)+0.001)
#mse = tf.reduce_mean(tf.squared_difference(out, Y))

In [47]:
# Optimizer
opt = tf.train.AdamOptimizer().minimize(mse)

In [48]:
import matplotlib.pyplot as plt

# Make Session
net = tf.Session()
# Run initializer
net.run(tf.global_variables_initializer())

# Setup interactive plot
plt.ion()
fig = plt.figure()
ax1 = fig.add_subplot(111)
line1, = ax1.plot(y_test)
line2, = ax1.plot(y_test*0.5)
plt.show()

# Number of epochs and batch size
epochs = 20
batch_size = 256

for e in range(epochs):
    #print(e,end=' ')
    # Shuffle training data
    shuffle_indices = np.random.permutation(np.arange(len(y_train)))
    X_train = X_train[shuffle_indices]
    y_train = y_train[shuffle_indices]

    # Minibatch training
    for i in range(0, len(y_train) // batch_size):
        start = i * batch_size
        batch_x = X_train[start:start + batch_size]
        batch_y = y_train[start:start + batch_size]
        # Run optimizer with batch
        net.run(opt, feed_dict={X: batch_x, Y: batch_y})

        # Show progress
        if np.mod(i, 5) == 0:
            # Prediction
            pred = net.run(out, feed_dict={X: X_test})
            line2.set_ydata(pred)
            plt.title('Epoch ' + str(e) + ', Batch ' + str(i))
            plt.pause(0.01)
    # Print final MSE after Training
    mse_train = net.run(mse, feed_dict={X: X_train, Y: y_train})
    mse_test = net.run(mse, feed_dict={X: X_test, Y: y_test})
    print(mse_train, mse_test)
    

nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
